{% include nav/toolkits/bathroom/menu.html %}

<div class="container">
    <div class="components">
        <!-- queue header and your info like ETA -->
        <div class="div1">
            <div class="queue-header">
                <h2 id="teacherName">Mr. Mortensen's Queue</h2>
            </div>

            <div class="queue-info">
                <p id="position">Your position in line: #</p>
                <p id="estimatedTime">Estimated time: ~ mins</p>
            </div>

            <hr>
        </div>

        <!-- students in line -->
        <div class="div2">
            <div class="queue-list">
                <p>Students currently in line:</p>
                <ul id="studentList">
                    <!-- Students will be listed here -->
                </ul>

                <div class="queue-buttons queue-actions">
                    <button type="button" onclick="window.addToQueue()">Add Me to Queue</button>
                    <button type="button" onclick="window.removeFromQueue()">Remove Me from Queue</button>
                    <button type="button" href="{{site.baseurl}}/bathroom/issues">Issues</button>
                </div>
            </div>
        </div>
    </div>
</div>
<!-- New "Request Pass" button at the bottom -->
<div class="bottom-button-container">
    <button class="big-button" onclick="window.location.href='{{site.baseurl}}/bathroom/requestpass'">
        Request Pass
    </button>
</div>

<style>
    .bottom-button-container {
        display: flex;
        justify-content: center;
        margin-top: 20px;
    }

    .big-button {
        font-size: 40px;
        padding: 15px 30px;
        background-color: #28a745;
        color: white;
        border: none;
        border-radius: 10px;
        cursor: pointer;
    }

    .big-button:hover {
        background-color: #218838;
    }
</style>


<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<script type="module" src="https://unpkg.com/ionicons@7.1.0/dist/ionicons/ionicons.esm.js"></script>
<script nomodule src="https://unpkg.com/ionicons@7.1.0/dist/ionicons/ionicons.js"></script>

<script type="module">
    import {javaURI, fetchOptions} from "{{site.baseurl}}/assets/js/api/config.js";
    const queueURI = javaURI + '/api/queue';
    const getUrl = queueURI + '/all';
    const addUrl = queueURI + '/add';
    const removeUrl = queueURI + '/remove';
    const approvalUrl = queueURI + '/approve';

    const postOptions = {
        ...fetchOptions,
        method: 'POST',
    };
    const deleteOptions = {
        ...fetchOptions,
        method: 'DELETE',
    };

    // Global variables
    const teacherName = "jm1021@gmail.com"; // Replace with dynamic teacher name if needed

    window.studentsInQueue = [];
    window.isApproving = false; // Prevent duplicate approve calls

    window.addEventListener('load', () => {
        fetchUser();
    });

    async function fetchUser() {
        const response = await fetch(javaURI + `/api/person/get`, {
            method: 'GET',
            cache: "no-cache",
            credentials: 'include',
            headers: { 
                'Content-Type': 'application/json',
                'X-Origin': 'client' 
            }
        });
        if (response.ok) {
            const userInfo = await response.json();
            window.studentName = userInfo.name;
            window.sid = userInfo.sid;
            window.roles = userInfo.roles.map(role => role.name).join(', ');
            console.log("Person: ", userInfo);
        }
    }

    // Display queue
    window.displayQueue = function() {
        const studentList = document.getElementById("studentList");
        studentList.innerHTML = ""; // Clear the list

        window.studentsInQueue.forEach((student, index) => {
            const listItem = document.createElement("li");

            // Indicate waiting for approval for the first student in line
            if (index === 0) {
                listItem.textContent = `${student} (waiting for approval)`;
            } else {
                listItem.textContent = student;
            }

            if (student === window.studentName) listItem.style.color = "red"; // Highlight user's name in red
            studentList.appendChild(listItem);
        });

        const addButton = document.querySelector('button[onclick="window.addToQueue()"]');
        addButton.disabled = window.studentsInQueue.includes(window.studentName);

        const position = window.studentsInQueue.indexOf(window.studentName) + 1;
        document.getElementById("position").textContent = `Your position in line: #${position}`;
        document.getElementById("estimatedTime").textContent = `Estimated time: ~${position * 3} mins`;
    }

    // Fetch queue data
    window.fetchQueueData = function() {
        fetch(getUrl, fetchOptions)
            .then(response => {
                if (response.status !== 200) {
                    console.error("Failed to fetch queue data.");
                    return;
                }
                return response.json();
            })
            .then(data => {
                const mortensenQueue = data.find(queue => queue.teacherEmail === teacherName);
                if (mortensenQueue) {
                    window.studentsInQueue = mortensenQueue.peopleQueue.split(',');
                    const isFrontOfQueue = window.studentsInQueue[0] === window.studentName;

                    if (isFrontOfQueue && !window.isApproving) {
                        if (mortensenQueue.away === 1 && isFrontOfQueue && !window.isApproving) {
                            window.location.href = "{{site.baseurl}}/bathroom/hallpass";
                            return;
                        }
                        window.approveStudent();
                    }
                }
                window.displayQueue();
            })
            .catch(error => console.error("Error fetching data:", error));
    }

    // Add student to queue
    window.addToQueue = function() {
        const queuer = {
            teacherEmail: teacherName,
            studentName: studentName,
            uri: javaURI,
        };
        if (window.studentsInQueue.includes(window.studentName)) {
            alert("You are already in the queue.");
            return;
        }

        fetch(addUrl, {
            ...postOptions,
            body: JSON.stringify(queuer),
        })
            .then(response => {
                if (response.ok) window.fetchQueueData();
                else alert("Failed to add to queue.");
            })
            .catch(error => console.error("Error adding to queue:", error));
    }

    // Remove student from queue
    window.removeFromQueue = function() {
        const queuer = {
            teacherEmail: teacherName,
            studentName: studentName,
            uri: javaURI,
        };
        fetch(removeUrl, {
            ...deleteOptions,
            body: JSON.stringify(queuer),
        })
            .then(response => {
                if (response.ok) {
                    window.fetchQueueData();
                } else alert("Failed to remove from queue.");
            })
            .catch(error => console.error("Error removing from queue:", error));
    }

    // Approve student at the front of the queue
window.approveStudent = async function () {
    // If already approving, return immediately
    if (window.isApproving) return;

    // Set the flag to true to prevent duplicate calls
    window.isApproving = true;

    try {
        // Capture the current time
        const now = new Date();
        const hours = now.getHours();
        const minutes = now.getMinutes().toString().padStart(2, "0");
        const seconds = now.getSeconds().toString().padStart(2, "0");
        const timeIn = `${hours}:${minutes}:${seconds}`;
        localStorage.setItem("timeIn", timeIn);

        // Prepare the payload
        const queuer = {
            teacherEmail: teacherName,
            studentName: studentName,
            uri: javaURI,
        };

        // Make the API call to approve the student
        const response = await fetch(approvalUrl, {
            ...postOptions,
            body: JSON.stringify(queuer),
        });

        if (!response.ok) {
            alert("Failed to approve student.");
        }
    } catch (error) {
        console.error("Error approving student:", error);
    } finally {
        // Reset the flag once the operation is complete
        window.isApproving = false;
    }
};

// Fetch data every 10 seconds
setInterval(window.fetchQueueData, 5000);


    // Initial fetch
    window.fetchQueueData();
</script>
